In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import norm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

pd.set_option('max_columns', 100)

# Visualization Libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import time
import copy

# torch things
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader


# Extras
import math, string, os
from string import punctuation
import re
import math
from scipy.spatial import Voronoi, voronoi_plot_2d

from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


/kaggle/input/nfl-big-data-bowl-2020/train.csv
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/test.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/sample_submission.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/__init__.py
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/competition.cpython-36m-x86_64-linux-gnu.so


In [2]:
import os
__print__ = print
def print(string):
    """
    Do this to print to kaggle log
    """
    os.system(f'echo \"{string}\"')
    __print__(string)

In [3]:
def process(df, categories, all_categories, test_or_train='train', convert_categorical=True, tokenizer=None):    
  for col in df.columns: 
    if df[col].dtypes == 'O':
      df[col].fillna("N/A", inplace=True)
    else: 
      df[col].fillna(df[col].mean(), inplace = True) 
  def clean_abbrevs(txt):
    if 'CLV' == txt:
      return 'CLE'
    elif 'BLT' == txt:
      return 'BAL'
    elif 'ARZ' == txt:
      return 'ARI'
    elif 'HST' == txt:
      return 'HOU'
    else: 
      return txt


  #StadiumDict = {'out': 'outdoor', 'open':'outdoor', 'in':'indoor','closed':'indoor', 'dome':'rtr roof'}
  def clean_StadiumType(txt):
    if pd.isna(txt):
          return 'N/A'
    txt = txt.lower()
    if 'out' in txt:
      return 'outdoor'
    elif 'open' in txt:
      return 'outdoor'
    elif 'in' in txt:
      return 'indoor'
    elif 'closed' in txt:
      return 'indoor'
    elif 'dome' in txt:
      return 'rtr roof'
    else:
      return 'N/A'


  def clean_Position(txt):
    if txt == 'S':
      txt = txt.replace('S', 'SA')
    if txt == 'SAF':
      txt = txt.replace('SAF', 'SA')
    if txt == 'HB':
      txt = txt.replace('HB', 'RB')
    if txt == 'FB':
      txt = txt.replace('FB', 'RB')
    if txt == 'MLB':
      txt = txt.replace('MLB', "ILB")
    if txt == 'NT':
      txt = txt.replace('NT', 'DT')
    if txt =='OG': 
      txt = txt.replace('OG', 'G')
    if txt =='OT': 
      txt = txt.replace('OT', 'T')
    return txt


  def clean_GameWeather(txt):
    if pd.isna(txt):
          return 'N/A'
    txt = txt.lower()
    if 'rain' in txt:
      return 'rain'
    elif 'snow' in txt:
      return 'snow'
    elif 'cloudy' in txt:
      return 'cloudy'
    elif 'sun' in txt:
      return 'clear'
    elif 'clear' in txt:
      return 'clear'
    elif 'in' in txt:
      return 'clear'
    else:
      return 'N/A'


  directions = ['n','nne', 'ne', 'ene', 'e', 'ese', 'se', 'sse', 's', 'ssw', 'sw', 'wsw', 'w', 'wnw', 'nw', 'nnw']
  def clean_WindDirection(txt):
    if pd.isna(txt):
          return 'N/A'
    txt = txt.lower()
    if txt in directions:
      return txt
    elif 'southwest' in txt:
      return 'sw'
    elif 'southeast' in txt:
      return 'se'
    elif 'northwest' in txt:
      return 'nw'
    elif 'northeast' in txt:
      return 'ne'
    elif 'north' in txt:
      return 'n'
    elif 'south' in txt:
      return 's'
    elif 'west' in txt:
      return 'w'
    elif 'east' in txt:
      return 'e'
    elif 'from' in txt:
      txt = txt.split(" ")
      return txt[1]
    elif '-' in txt:
      txt = txt.split("-")
      return txt[0]+txt[1]
    elif 'calm' in txt:
      return 'calm'
    else:
      return 'N/A'


  def clean_Turf(txt):
      if pd.isna(txt):
          return np.nan
      txt = txt.lower()
      if 'DD' in txt:
        return 'Hybrid'
      elif 'SIS' in txt:
        return 'Hybrid'
      elif 'grass' in txt:
        return 'Grass'
      elif 'natural' in txt:
        return 'Grass'
      else:
        return 'Turf'

  def convert_heights(txt):
    ft, inch = txt.split("-")
    return int(ft)*12 + int(inch)

  def convert_birthday(txt):
    month, day, year = txt.split('/')
    return ((int(month) - 1)*30 + int(day) + (2019-int(year))*365)/365


  def convert_time(txt1, txt2):
    date, time = txt1.split('T')
    time, z = time.split('Z')
    hours, mins, secs = time.split(':')

    date2, time2 = txt2.split('T')
    time2, z2 = time2.split('Z')
    hours2, mins2, secs2 = time2.split(':')

    dhours = float(hours2) - float(hours)
    dmins = float(mins2) - float(mins)
    dsecs = float(secs2) - float(secs)
    dtime = dhours * 3600 + dmins * 60 + dsecs
    if dtime < 0:
      return 86400+dtime
    else: 
      return dtime

  def convert_yardline(poss_team, side, yards):
    try: 
      if side == poss_team:
        return int(yards)
      else:
        return -50+int(yards)
    except: 
      return 50.0
    
  def convert_time_on_clock(quarter, game_clock):
    minutes, seconds, a = game_clock.split(':')
    time_remaining = (int(quarter))*15*60-int(minutes)*60-int(seconds)
    return time_remaining

  def windspeed_fix(txt):
    txt = str(txt)
    try: 
      speed = float(txt.split(' ')[0])
      return speed
    except: 
      return 0.0

  def direction_fix(df):
    if df.PlayDirection.iloc[0] == 'left':
      df.X = 120 - df.X
      df.Y = 53.33333 - df.Y
      df.Orientation = (180 + df.Orientation)%360.
      df.Dir = (180 + df.Dir)%360.
    return df

  print("Cleaning data")
  df['PossessionTeam'] = df['PossessionTeam'].apply(clean_abbrevs)
  df['FieldPosition'] = df['FieldPosition'].apply(clean_abbrevs)

  # clean turf
  df['Turf'] = df['Turf'].apply(clean_Turf)
  Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 
          'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 
          'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
          'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 
          'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
          'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 
          'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 
          'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
  df['Turf'] = df['Turf'].map(Turf)

  df['WindDirection'] = df['WindDirection'].apply(clean_WindDirection)

  # clean game weather
  indoor = "indoor"
  df['GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
  df['GameWeather'] = df['GameWeather'].apply(lambda x: x.lower().replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly').replace('clear and sunny', 'sunny and clear').replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
  df['GameWeather'] = df['GameWeather'].apply(clean_GameWeather)
    
  df['StadiumType'] = df['StadiumType'].apply(clean_StadiumType)
  df.loc[df.Season == 2017, 'Orientation'] = np.mod(90 + df.loc[df.Season == 2017, 'Orientation'], 360)
  df["PlayerHeight"] = df["PlayerHeight"].apply(convert_heights)
  df["WindSpeed"] = df["WindSpeed"].apply(windspeed_fix)
  df['PlayerAge'] = df['PlayerBirthDate'].apply(convert_birthday)
  df['Position'] = df['Position'].apply(clean_Position)

  array=[]
  for i in range(df.shape[0]): 
    array.append(convert_time(df['TimeSnap'][i],df['TimeHandoff'][i]))
  df['TimeToHandoff'] = array

  array=[]
  for i in range(df.shape[0]): 
    array.append(convert_yardline(df['PossessionTeam'][i], df['FieldPosition'][i],df['YardLine'][i]))
  df['ScrimmageLine'] = array

  array=[]
  for i in range(df.shape[0]): 
    array.append(convert_time_on_clock(5 - df['Quarter'][i], df['GameClock'][i]))
  df['Time'] = array

  array=[]
  for i in range(df.shape[0]): 
    array.append(convert_time_on_clock(1, df['GameClock'][i]))
  df['QuarterTime'] = array

  df = df.groupby("PlayId", group_keys=False).apply(direction_fix)


  def add_positions(df):
    off_pos = ''.join(df.OffensePersonnel.unique())
    off_pos = np.unique(re.findall(r'\b[A-Z][A-Z]+\b', off_pos))
    off_pos = ['Off' + p for p in off_pos]
    
    def_pos = ''.join(df.DefensePersonnel.unique())
    def_pos = np.unique(re.findall(r'\b[A-Z][A-Z]+\b', def_pos))
    def_pos = ['Def' + p for p in def_pos]
  
    df = df.reindex(columns=df.columns.values.tolist() + off_pos + def_pos)
  
    def parse_df(df):
      off_ = df.iloc[0].OffensePersonnel.replace(',', ' ').split()
      nums, cols = off_[::2], off_[1::2]
      cols = ['Off' + c for c in cols]
      df[cols] = nums
      
      def_ = df.iloc[0].DefensePersonnel.replace(',', ' ').split()
      nums, cols = def_[::2], def_[1::2]
      cols = ['Def' + c for c in cols]
      df[cols] = nums
      return df
    
    df = df.groupby('PlayId').apply(parse_df)
    df.drop(columns=['OffensePersonnel', 'DefensePersonnel'], inplace=True)

    for c in off_pos + def_pos:
      df[c] = df[c].fillna(0)
      
    return df

  print("adding positions")
  df = add_positions(df)

  
  def make_vectors(points):
    """
    Take a collection of vertices (points) of a convex shape
    and create a set of vectors from the uppermost point
    to the rest of the points
    """
    points = np.array(sorted(points, key=lambda x: -x[1]))
    vecs = (points - points[0])[1:]
    return vecs

  def order_vectors(vectors):
      """
      top: top of polygon from which all vectors are defined
      vectors: (x, y) of vectors defined by make_vectors
      """
      def angle(vec):
          xhat = np.array([1, 0])
          sin_angle = np.linalg.norm(np.cross(xhat, vec))/np.linalg.norm(vec)
          angle = np.arcsin(sin_angle)
          if vec[0] < 0:
              # because arcsin only gives between -pi/2 and pi/2
              angle = np.pi - angle
          return angle
      
      return sorted(vectors, key=lambda vec: angle(vec))

  def polygon_area(points):
      """
      We break the polygon into triangles and then sum up
      the areas of those individual triangles to determine
      the total area of the polygon. This only works because
      the polygons are convex
      """
      vectors = make_vectors(points)
      ord_vecs = order_vectors(vectors)
      
      area = 0
      i = 0
      while i < len(ord_vecs)-1:
          area += 0.5 * np.linalg.norm(np.cross(ord_vecs[i], ord_vecs[i+1]))
          i += 1
      
      return area

  def get_dx_dy(delta_t, rusher_dir, rusher_or, speed, acceleration):
      dx = (delta_t*speed) * math.cos(rusher_dir)
      dx = dx + 0.5*delta_t**2 * acceleration * math.cos(rusher_or)
      dy = (delta_t*speed) * math.sin(rusher_dir)
      dy = dy + 0.5*delta_t**2 * acceleration * math.sin(rusher_or)
      return dx, dy

  def voronoi_dist_to_rb(df, delta_t):
        """Compute Voronoi areas and distance to RB for an array of Δt offsets."""
        try:
          iter(delta_t)
        except:
          delta_t = [delta_t]

        direction = np.mod(90. - df.Dir, 360.) * np.pi / 180.
        orientation = np.mod(90. - df.Orientation, 360.) * np.pi / 180.

        for dt in delta_t:
          dx = (dt * df.S) * np.cos(direction) + 0.5 * dt ** 2 * df.A * np.cos(orientation)
          dy = (dt * df.S) * np.sin(direction) + 0.5 * dt ** 2 * df.A * np.sin(orientation)
          x_ = df.X + dx; y_ = df.Y + dy
          df['X_' + str(dt)] = x_
          df['Y_' + str(dt)] = y_

        def add_features(df):
          for dt in delta_t:
              # Compute Voronoi areas
              coords = ['X_' + str(dt), 'Y_' + str(dt)]
              vor = Voronoi(df[coords].values)
              areas = np.array([polygon_area([vor.vertices[i] for i in region])
                              if len(region) > 0 else 0. for region in vor.regions])
              df['VoronoiArea_' + str(dt)] = areas[vor.point_region]

              # Compute distance to RB
              rusher = df[df.NflId == df.NflIdRusher]
              dist = np.linalg.norm((df[coords] - rusher[coords].values).values, axis=1)
              df['DistanceToRB_dt_' + str(dt)] = dist
          return df

        df = df.groupby('PlayId', group_keys=False).apply(add_features)
        df = df.drop(columns=['X_' + str(dt) for dt in delta_t] + ['Y_' + str(dt) for dt in delta_t])
        return df

  delta_t = [0., 0.25, 0.5, 0.75]
  print("adding vorobois")
  df = voronoi_dist_to_rb(df, delta_t)

  def centroid_dif(df): 
    homeTeam = df[df.Team == 'home']
    awayTeam = df[df.Team == 'away']
    centroid = np.sqrt((homeTeam.X.mean()-awayTeam.X.mean())* (homeTeam.X.mean()-awayTeam.X.mean()) 
                    +(homeTeam.Y.mean()-awayTeam.Y.mean())*(homeTeam.Y.mean()-awayTeam.Y.mean()))
    df["Centroid"] = np.ones((22,))*centroid
    return df

  print("Adding Centroid Diffs")
  df = df.groupby('PlayId', group_keys=False).apply(centroid_dif)

  df['MinTimeToTackle']= df['DistanceToRB_dt_0.0']/df.S

  direction = np.mod(90. - df.Dir, 360.) * np.pi / 180.0
  orientation = np.mod(90. - df.Orientation, 360.) * np.pi / 180.0
  df['Sx'] =  (df.S) * np.cos(direction)
  df['Sy'] = (df.S) * np.sin(direction)


  def nn_to_rb(df):
    """Get nearest neighbors to running back"""
    is_defense = (~np.any(df.NflId == df.NflIdRusher)).astype('int')
    df = df.sort_values('DistanceToRB_dt_0.0')
    df['NNToRB'] = is_defense * 11 + np.arange(11)
    return df

  print("NN to Rusher")
  df = df.groupby(['PlayId', 'Team'], group_keys=False).apply(nn_to_rb)


  print("Converting categorical") 
  if convert_categorical:
    cat_df = df[categories]
    if test_or_train == 'train':
      # tokenize categorical variables
#       categories = ['PossessionTeam', 'Down', 'NflIdRusher', 'OffenseFormation', 
#                   'NflId', 'JerseyNumber', 'Position', 
#                   'PlayerHeight', 'PlayerWeight','PlayerCollegeName',
#                   'HomeTeamAbbr', 'VisitorTeamAbbr', 'Week', 'Stadium', 'Location', 
#                   'StadiumType', 'Turf', 'GameWeather', 'Temperature', 'Humidity', 
#                   'WindSpeed', 'WindDirection',  'Quarter'] 

      cat_df = cat_df.astype('category')
      cat_df = cat_df.apply(lambda x: x.cat.codes)

      def create_tokenizer(df, cat_df):
        tokenizer = {}
        for cat in categories:
            a = df[cat].unique()
            b = cat_df[cat].unique()
            tokenizer.update({cat: {num: tok for num, tok in zip(a, b)}})
        return tokenizer
    
      print("Making tokenizer")
      tokenizer = create_tokenizer(df, cat_df)
    elif test_or_train == 'test':
        for cat in categories:
          cat_df[cat] = df[cat].map(tokenizer[cat])     
        
        # note: if the dictionary does not exhaustively map all
        # entries then non-matched entries are changed to NaNs
        
    df[categories] = cat_df


  def flatten_data(df): 
    """Flatten grouped dataframe and return that dataframe""" 
    df = df.sort_values(by = ['NNToRB'])
    array = df.values.flatten()
    array = np.transpose(array.reshape(array.shape[0], 1))
    df = pd.DataFrame(data = array, columns = playerDataCol)
    return df

  def game_data(df):
    df = df.iloc[0,:]
    array = df.values.flatten()
    array = np.transpose(array.reshape(array.shape[0], 1))
    df = pd.DataFrame(data = array, columns = gameData)
    return df

  """We need a better way of picking out the columns related to players vs. games 
    although I guess doing it manually isn't that bad.... """

  gameData = ['GameId', 'PlayId', 'Team', 'Season', 'YardLine',
        'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
        'FieldPosition', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
        'NflIdRusher', 'OffenseFormation', 'DefendersInTheBox', 'PlayDirection',
        'TimeHandoff', 'TimeSnap', 'Yards', 'HomeTeamAbbr',
        'VisitorTeamAbbr', 'Week', 'Stadium', 'Location', 'StadiumType', 'Turf',
        'GameWeather', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection',
        'ScrimmageLine', 'Time']
  if test_or_train == 'test':
        gameData.remove('Yards')

  playerData=['PlayId','X', 'Y', 'S', 'A', 'Dis', 'Orientation',
        'Dir', 'NflId', 'DisplayName', 'JerseyNumber', 'PlayerHeight', 'PlayerWeight',
        'PlayerBirthDate', 'PlayerCollegeName', 'Position', 
        'PlayerAge', 'Centroid', 'Sx', 'Sy',
        'DistanceToRB_dt_0.0', 'DistanceToRB_dt_0.25', 'DistanceToRB_dt_0.5',
        'DistanceToRB_dt_0.75','VoronoiArea_0.0','VoronoiArea_0.25',
        'VoronoiArea_0.5','VoronoiArea_0.75',  'NNToRB']
        

  playerDataCol=[]
  sides = ['_O_', '_D_']
  for k,side in enumerate(sides): 
    for i in range(11): 
      for player_values in playerData: 
        title = str(player_values)+side+str(i+k)
        playerDataCol.append(title)

  dfPlayers = df[playerData]
  dfGame = df[gameData]

  print("Flatten data")
  dfPlayers = dfPlayers.groupby(['PlayId'], group_keys=False).apply(flatten_data)
  dfGame = dfGame.groupby(['PlayId'], group_keys=False).apply(game_data)

  df = pd.concat([dfGame, dfPlayers], axis = 1)
  df['HandoffToFD'] = df['Distance'] + (df['X_O_0']-df['ScrimmageLine'])


  df = df.replace([np.inf, -np.inf], np.nan)
  df = df.replace(np.nan, 10)
  
  if convert_categorical:
        return df, tokenizer
  return df

In [4]:
eps = 1e-10

class FCEmbed(nn.Module):
    def __init__(self, in_dim, out_dim, vocab_sizes, vector_sizes,
                 depth=1, width=500, dropout_rate=0.1):
        """
        Mama mia
        """
        super(FCEmbed, self).__init__()
        in_dim = in_dim + sum(vector_sizes)

        self.in_dim = in_dim 
        self.out_dim = out_dim
        self.vocab_sizes = vocab_sizes
        self.vector_sizes = vector_sizes
        self.depth = depth
        self.width = width
        self.dropout_rate = dropout_rate

        # build model layers
        mid_layer = [nn.Linear(width, width), nn.ReLU(), nn.Dropout(dropout_rate)]

        all_layers = [nn.Linear(in_dim, width), nn.ReLU()]
        for d in range(depth):
            all_layers.extend(mid_layer)
        all_layers.append(nn.Linear(width, out_dim))
        all_layers.append(nn.Softmax(dim=1))

        # vocab_size is the number of words in your train, val and test set
        # vector_size is the dimension of the word vectors you are using. 
        #   # this is the number of categorical variables?
        self.embed = Embeds(vocab_sizes, vector_sizes)
        self.model = nn.Sequential(*all_layers)

    def forward(self, x_num, x_cat):
        # Then turn the word index into actual word vector
        x_cat = self.embed(x_cat)

        # there needs to be a better way...
        x = torch.cat([x_num.transpose(0, 1), x_cat.transpose(0, 1)]).transpose(0, 1)

        out = self.model(x)
        out = torch.clamp(out.cumsum(1), min=eps, max=1 - eps)
        return out

    def loss(self, guess, label, device):
        L = F.mse_loss(guess, label)

        return L

class Embeds(nn.Module):
    def __init__(self, vocab_sizes, vector_sizes, depth=1, width=500, dropout_rate=0.1):
        """
        Mama mia
        vocab_sizes: list of sizes of vocabs for each categorical column
        vector_size: list of output vector dims for each categorical column
        """
        super(Embeds, self).__init__()
        self.vocab_sizes = vocab_sizes
        self.vector_sizes = vector_sizes
        # vocab_size is the number of words in your train, val and test set
        # vector_size is the dimension of the word vectors you are using. 
        self.embeds = nn.ModuleList([nn.Embedding(voc_size, vec_size) for voc_size, vec_size in zip(vocab_sizes, vector_sizes)])

    def forward(self, all_x):
        out = []
        for i, x in enumerate(all_x.transpose(0, 1)):
            x = abs(x) # negative indices bad
            out.append(self.embeds[i].forward(x).transpose(0, 1))
        out = torch.cat(out).transpose(0, 1)
        return out


def train(model, device, train_loader, optimizer, epoch, log_interval=1e5, verbose=True):
    model.train()
    train_loss = 0

    for batch_idx, (data_num, data_cat, yardage) in enumerate(train_loader):
        data_num = data_num.to(device)
        data_cat = data_cat.to(device)
        optimizer.zero_grad()
        output = model(data_num, data_cat)

        loss = model.loss(output, yardage, device)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if batch_idx % log_interval == 0:
            if verbose:
                print('{} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    time.ctime(time.time()), epoch, batch_idx * len(data_num),
                    len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

    train_loss /= len(train_loader)
    if verbose:
        print('Train set Average loss: {}'.format(train_loss))
    return train_loss

def label_to_cdf(label, noisy=None):
    """
    labels are the yards gained. thus the cdf is a step function at yards gained

    noisy: treat yards as a gaussian centered on label with sigma = noisy
    """
    label = label + 99 # to adjust for negative yards
    if noisy:
        # noisy = number of bins to noise over
        bins = [199 - i for i in range(199)]
        cdf = norm.cdf(bins, 199 - label, noisy)
    else:
        cdf = np.sum(np.eye(199)[:label], axis=0) 
    return 1 - cdf

def test(model, device, test_loader, log_interval=1e5, verbose=True, return_predictions=False):
    model.eval()
    test_loss = 0

    all_predictions = []

    with torch.no_grad():
        for batch_idx, (data_num, data_cat, yardage) in enumerate(test_loader):
            data_num = data_num.to(device)
            data_cat = data_cat.to(device)
            output = model(data_num, data_cat)
            all_predictions.append(output)
            loss = model.loss(output, yardage, device)
            test_loss += loss.item()

            if log_interval is not None and batch_idx % log_interval == 0:
                if verbose:
                    print('{} Test: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        time.ctime(time.time()),
                        batch_idx * len(data_num), len(test_loader.dataset),
                        100. * batch_idx / len(test_loader), loss.item()))

    test_loss /= len(test_loader)
    if verbose:
        print('Test set Average loss: {}'.format(test_loss))

    if return_predictions:
        return all_predictions, test_loss
    return test_loss


class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, data_num, data_cat, labels):
        'Initialization'
        self.data_num = data_num
        self.data_cat = data_cat
        self.labels = labels

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data_num)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        X_num = self.data_num[index]
        X_cat = self.data_cat[index]
        y = self.labels[index]
        return X_num, X_cat, y

In [5]:
xTrainMin = -7777777
def prep_data(df, categories, all_categories):
    df = df[df.Yards < BIG_YARDCUTOFF].reset_index()
    df = df[df.Yards > SMALL_YARDCUTOFF].reset_index()
    
    xTrain, tokenizer = process(df, categories, all_categories,
                                test_or_train='train', convert_categorical=True)
    
    x_cat = xTrain[categories].values
    df = xTrain.drop(all_categories, axis=1)
    print("Done processing.")
#     df2018 = xTrain[xTrain['Season']==2018]
#     df2017 = xTrain[xTrain['Season']==2017]
#     df2017['Yards'] = 4.235*df2017['Yards']/df2017['Yards'].mean()
#     df2018['Yards'] = 4.235*df2018['Yards']/df2018['Yards'].mean()
#     dfy = df2017.append(df2018)
#   yTrain = dfy['Yards']
    yTrain = xTrain['Yards']
    
    xTrain = xTrain.drop(['Season'], axis=1)
    # drop all PlayIDs
    xTrain = xTrain.infer_objects()
    xTrain = xTrain.drop([col for col in xTrain.columns if 'PlayId' in col], axis=1)
    xTrain = xTrain.drop([col for col in xTrain.columns if 'NflId' in col], axis=1)
    xTrain = xTrain.select_dtypes(exclude=['object'])
    xTrain = xTrain.drop(['GameId', 'Yards'], axis=1)
    
    xTrain = xTrain[[col for col in xTrain.columns if ('Voronoi' in col or 'DistanceToRB' in col or 'X_' in col or 'Y' in col or 'S_' in col or 'A_' in col)]]

    xTrain = np.log(xTrain - xTrainMin + eps) # to bring things much closer
    x = xTrain
    
    # set up labels
    y = np.array([label_to_cdf(l, noisy=NOISY) for l in yTrain])
    y = y.astype('float32')

    return x, x_cat, y, tokenizer

def train_model(x, x_cat, y, width=128, scaler=None, pca=None): 
    # standard scaler
    if scaler == None:
        scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)


    # PCA preprocessing
    if pca == None:
        pca = PCA(n_components=max(20, x.shape[-1]//2))
        # pca = FunctionTransformer()
    pca.fit(x)
    x = pca.transform(x)

    x = x.astype('float32')

    # set up labels
    y = y.astype('float32')

    # make validation set
    msk = np.random.rand(len(x)) < 0.8
    msk = int(len(x)*0.8)

    x_num_train = x[:msk]
    x_cat_train = x_cat[:msk]
    y_train = y[:msk]

    x_num_val = x[msk:]
    x_cat_val = x_cat[msk:]
    y_val = y[msk:]

    # x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

    train_loader = DataLoader(Dataset(x_num_train, x_cat_train, y_train), 
                              batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(Dataset(x_num_val, x_cat_val, y_val), 
                              batch_size=BATCH_SIZE, shuffle=True)
    # make model
    IN_DIM = x.shape[-1]
    OUT_DIM = 199
    model = FCEmbed(IN_DIM, OUT_DIM, VOCAB_SIZES, VECTOR_SIZES,
                    dropout_rate=DROPOUT_RATE, width=WIDTH)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # train nn
    best_loss = 1e10
    best_epoch = 0
    early_stopping = 30
    for epoch in range(0, EPOCHS + 1):
        train_loss = train(model, device, train_loader, optimizer, 
                                      epoch, verbose=True)
        valid_loss = test(model, device, valid_loader, verbose=True)

        if valid_loss < best_loss:
            print("victory")
            best_loss = valid_loss
            best_epoch = epoch
            best_model = copy.deepcopy(model)
        
        if epoch - best_epoch > early_stopping:
            break

    return best_model, pca, scaler
    

def predict(ensemble, test_df, categories, all_categories, tokenizer, clip_min=-1e10, clip_max=1e10, 
            pca=FunctionTransformer, scaler=FunctionTransformer):
    
    playId = test_df['PlayId']
    xTest, tokenizer = process(test_df, categories, all_categories, 
                    test_or_train='test', convert_categorical=True, tokenizer=tokenizer)
    
    x_cat = xTest[categories].values
    df = xTest.drop(all_categories, axis=1)
    
    xTest = xTest.drop(['Season'], axis=1)
    # drop all PlayIDs
    xTest = xTest.infer_objects()
    xTest = xTest.drop([col for col in xTest.columns if 'PlayId' in col], axis=1)
    xTest = xTest.drop([col for col in xTest.columns if 'NflId' in col], axis=1)
    xTest = xTest.select_dtypes(exclude=['object'])
    xTest = xTest.drop(['GameId'], axis=1)
    
    xTest = xTest[[col for col in xTest.columns if ('Voronoi' in col or 'DistanceToRB' in col or 'X_' in col or 'Y' in col or 'S_' in col or 'A_' in col)]]

    xTest = np.log(xTest - xTrainMin + eps) # to bring things much closer
    x = xTest 

    # clip large outlier values
    x = x.clip(clip_min, clip_max, axis=1)

    # std scaling
    x = scaler.transform(x)

    # PCA preprocessing
    x = pca.transform(x)
    x = x.astype('float32')
    
    data = torch.tensor(x)
    data_cat = torch.tensor(x_cat)
#     print(data)
    all_cdfVals = []
    for model in ensemble:
        model.eval()
        all_cdfVals.append(model(data, data_cat).detach())
    cdfVals = torch.stack(all_cdfVals).mean(0)
    
    submission_cols = np.array(['Yards'+str(i) for i in range(-99,100)])
    submission_df = pd.DataFrame(index=[test_df['PlayId'].iloc[0]], columns=submission_cols)

    submission_df.iloc[0]= np.clip(cdfVals, 0, 1)
    # submission df corresponds to just a single PlayId
    return submission_df



In [6]:
all_categories = ['PossessionTeam', 'Down', 'NflIdRusher', 'OffenseFormation', 
              'HomeTeamAbbr', 'VisitorTeamAbbr', 'Week', 'Stadium', 'Location', 
              'StadiumType', 'Turf', 'GameWeather', 
            #   'Temperature', 'Humidity', 
              'WindSpeed', 'WindDirection',  'Quarter']


player_cats = ['NflId_', 
#                'JerseyNumber_', 'Position_', 
              'PlayerHeight_', 'PlayerWeight_','PlayerCollegeName_'] 
full_player_cats = [cat+tm+"_"+str(nn+i) for i, tm in enumerate(["O", "D"]) for nn in range(11) for cat in player_cats]

all_categories.extend(full_player_cats)

categories = [
            #   'NflIdRusher', 
              'OffenseFormation',  
            #   'StadiumType', 
              'Turf', 
              'GameWeather', 
            #   'Temperature', 
            #   'Humidity', 
              'WindSpeed', 
            #   'WindDirection',  
              'Quarter',
            #   'NflId_O_0', 
#               'JerseyNumber', 
#               'Position', 
            #   'PlayerHeight_O_0', 
            #   'PlayerWeight_O_0',
            #   'PlayerCollegeName_O_0'
              ]




In [7]:
from kaggle.competitions import nflrush
env = nflrush.make_env()

# Training data is in the competition dataset as usual
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
# define model


In [8]:
# parameters
BATCH_SIZE = 1024
TEST_BATCH_SIZE = 10*BATCH_SIZE
EPOCHS = 100
DROPOUT_RATE = 0.5
DEPTH = 5
WIDTH = 64
LEARNING_RATE = 1e-3
NOISY = 0.0

# YARD CUTOFFS
BIG_YARDCUTOFF = 300
SMALL_YARDCUTOFF = -80

# CATEGORICAL STUFF
# VOCAB_SIZES = train_df[categories].max().values + 1
# VECTOR_SIZES = 2*np.sqrt(VOCAB_SIZES).astype(int) + 1
VOCAB_SIZES = [9, 2, 5, 24, 5]
VECTOR_SIZES = [7, 3, 5, 9, 5]

# cuda stuff
USE_CUDA = False
use_cuda = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [9]:
print("Start training")

Start training


In [10]:
begin_time = time.time()
ensemble = []
n_models = 10
x, x_cat, y, tokenizer = prep_data(train_df, categories, all_categories)
# x, x_cat, y, tokenizer = prep_data(train_df.iloc[:22*150], categories, all_categories)

Cleaning data
adding positions


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:260: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:255: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


adding vorobois


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:292: RuntimeWarning: invalid value encountered in double_scalars


Adding Centroid Diffs
NN to Rusher
Converting categorical
Making tokenizer
Flatten data
Done processing.


In [11]:

scaler = None
pca = None
for i in range(n_models):
    rand_width = np.random.randint(25, 1000)
    model, pca, scaler = train_model(x, x_cat, y, width=rand_width, scaler=scaler, pca=pca)
    ensemble.append(model)
    
print(time.time() - begin_time)

Tue Nov 26 02:03:21 2019 Train Epoch: 0 [0/18536 (0%)]	Loss: 0.084103
Train set Average loss: 0.07396467186902698
Tue Nov 26 02:03:22 2019 Test: [0/4635 (0%)]	Loss: 0.055744
Test set Average loss: 0.05548897460103035
victory
Tue Nov 26 02:03:22 2019 Train Epoch: 1 [0/18536 (0%)]	Loss: 0.055615
Train set Average loss: 0.034777455816143436
Tue Nov 26 02:03:23 2019 Test: [0/4635 (0%)]	Loss: 0.017486
Test set Average loss: 0.019000422582030295
victory
Tue Nov 26 02:03:23 2019 Train Epoch: 2 [0/18536 (0%)]	Loss: 0.018793
Train set Average loss: 0.016165589031420256
Tue Nov 26 02:03:24 2019 Test: [0/4635 (0%)]	Loss: 0.014403
Test set Average loss: 0.01508594211190939
victory
Tue Nov 26 02:03:24 2019 Train Epoch: 3 [0/18536 (0%)]	Loss: 0.015301
Train set Average loss: 0.014750363550295955
Tue Nov 26 02:03:25 2019 Test: [0/4635 (0%)]	Loss: 0.017324
Test set Average loss: 0.014690773747861385
victory
Tue Nov 26 02:03:25 2019 Train Epoch: 4 [0/18536 (0%)]	Loss: 0.015188
Train set Average loss: 0

In [12]:
print("Training done. Start predicting.")

Training done. Start predicting.


In [13]:
# break
for i, (test_df, sample_prediction_df) in enumerate(env.iter_test()):
    predictions_df = predict(ensemble, test_df.reset_index(), categories, all_categories, 
                         tokenizer, pca=pca, scaler=scaler)
    env.predict(predictions_df)
    print("Rough prediction: {}".format((predictions_df.values[0][1:] < 0.5).argmin()))
    
print("Done predicting. Writing submissions.")
env.write_submission_file()

Cleaning data
adding positions
adding vorobois
Adding Centroid Diffs
NN to Rusher
Converting categorical


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Flatten data
Rough prediction: 102
Cleaning data
adding positions
adding vorobois
Adding Centroid Diffs
NN to Rusher
Converting categorical


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Flatten data
Rough prediction: 102
Cleaning data
adding positions
adding vorobois
Adding Centroid Diffs
NN to Rusher
Converting categorical
Flatten data


...

NN to Rusher
Converting categorical
Flatten data


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Rough prediction: 103
Cleaning data
adding positions
adding vorobois
Adding Centroid Diffs
NN to Rusher
Converting categorical
Flatten data


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Rough prediction: 102
Done predicting. Writing submissions.
Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.


In [14]:
print("ALL DONE")

ALL DONE
